#### Importing Libraries

In [ ]:
# Install libraries
!pip install numpy pandas matplotlib sktime tqdm tensorflow scikit-image openpyxl

In [ ]:
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sktime.datasets import load_from_tsfile_to_dataframe
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing import image

#### Load Dataset

In [ ]:
dataset_dir = "/Users/amanmaurya/Desktop/TSC/Datasets" # Change the path to folder containing datasets.
dataset_name = "BasicMotions" # Change this to the dataset name
train_file = os.path.join(dataset_dir, dataset_name, f"{dataset_name}_TRAIN.ts")
test_file = os.path.join(dataset_dir, dataset_name, f"{dataset_name}_TEST.ts")

if not os.path.exists(train_file) or not os.path.exists(test_file):
    raise FileNotFoundError(f"Dataset files not found at {dataset_dir}. Check dataset path and structure.")

X_train, y_train = load_from_tsfile_to_dataframe(train_file)
X_test, y_test = load_from_tsfile_to_dataframe(test_file)


#### Characteristics of Loaded Dataset

In [ ]:
summary_data = {
    "Dataset": [],
    "Train size": [],
    "Test size": [],
    "Length": [],
    "Number of classes": [],
    "Number of dimensions": []
}

# Gather dataset properties
train_size = X_train.shape[0]
test_size = X_test.shape[0]
sample_lengths = [len(X_train.iloc[i, 0]) for i in range(train_size)]
length = sample_lengths[0]
num_classes = len(np.unique(y_train))
num_dimensions = X_train.shape[1]

# Append to summary table
summary_data["Dataset"].append(dataset_name)
summary_data["Train size"].append(train_size)
summary_data["Test size"].append(test_size)
summary_data["Length"].append(length)
summary_data["Number of classes"].append(num_classes)
summary_data["Number of dimensions"].append(num_dimensions)

# Convert summary to DataFrame and display
summary_df = pd.DataFrame(summary_data)
print(summary_df)

In [ ]:
# Define the base directory for saving generated images
output_dir = "Generated_Images/BasicMotions" # Change the name to the name of currently working dataset.

#### Helper Functions

In [ ]:
# Helper function to create directories
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Function to save an image for a given instance and dimension
def save_image(img, variant, dataset_type, label, instance_idx, dimension_idx):
    path = os.path.join(output_dir, variant, dataset_type, f"Class_{label}")
    create_dir(path)
    file_name = os.path.join(path, f"instance_{instance_idx}_dim_{dimension_idx}.png")
    plt.imsave(file_name, img, cmap='gray')

# Function to perform scaling to a target range
def scale_to_range(data, target_min, target_max):
    min_val = np.min(data)
    max_val = np.max(data)
    return target_min + (data - min_val) * (target_max - target_min) / (max_val - min_val)

#### Transformation methods

In [ ]:
def generate_images(X, y, dataset_type, method):
    for instance_idx, (instance, label) in enumerate(zip(X.iterrows(), y)):
        instance_idx, features = instance  # features is a pandas Series where each cell is a pandas.Series
        for dim_idx, ts_values in enumerate(features):  # Iterate over each feature (dimension)
            ts_values = np.array(ts_values)  # Convert pandas.Series to numpy array
            
            if method == "Variant 1":
                # Val/ValChng Transformation
                first_diff = np.diff(ts_values)
                scaled_values = scale_to_range(ts_values[1:], 0, 127)
                scaled_diff = scale_to_range(first_diff, 0, 127)
                img = np.zeros((128,128))
                for i, j in zip(scaled_values.astype(int), scaled_diff.astype(int)):
                    img[i, j] = min(img[i, j] + 1, 255)
                save_image(img, method, dataset_type, label, instance_idx, dim_idx)

            elif method == "Variant 2":
                # ValChng/ChngValChng Transformation
                first_diff = np.diff(ts_values)
                second_diff = np.diff(first_diff)
                scaled_first_diff = scale_to_range(first_diff[1:], 0, 127)
                scaled_second_diff = scale_to_range(second_diff, 0, 127)
                img = np.zeros((128,128))
                for i, j in zip(scaled_first_diff.astype(int), scaled_second_diff.astype(int)):
                    img[i, j] = min(img[i, j] + 1, 255)
                save_image(img, method, dataset_type, label, instance_idx, dim_idx)

            elif method == "Variant 3":
                # Values x Values Transformation
                scaled_values = scale_to_range(ts_values, 0, 1)
                img = np.outer(scaled_values, scaled_values)
                save_image(img, method, dataset_type, label, instance_idx, dim_idx)

            elif method == "Variant 4":
                # ReplVal Transformation
                scaled_values = scale_to_range(ts_values, 0, 1)
                img = np.tile(scaled_values, (len(ts_values), 1))
                save_image(img, method, dataset_type, label, instance_idx, dim_idx)

            elif method == "Variant 5":
                # ReplValChng Transformation
                first_diff = np.diff(ts_values)
                scaled_diff = scale_to_range(first_diff, 0, 1)
                img = np.tile(scaled_diff, (len(first_diff), 1))
                save_image(img, method, dataset_type, label, instance_idx, dim_idx)

#### Main loop to generate images

In [ ]:
methods = ["Variant 1", "Variant 2", "Variant 3", "Variant 4", "Variant 5"]
for method in methods:
    for dataset_type, (X, y) in zip(["Train", "Test"], [(X_train, y_train), (X_test, y_test)]):
        generate_images(X, y, dataset_type, method)

#### Load Images from Generated Images

In [ ]:
# Define image size for all variants
IMG_SIZE = (128,128)  # You can adjust this size

# Directory containing your images
image_dir = '/Users/amanmaurya/Desktop/TIme-Series-Classification-Via-Image-Transformations/Generated_Images/BasicMotions' # Change to the path of currently working generated images.

# Helper function to load and preprocess images for each variant
def load_images(image_dir, method, dataset_type, IMG_SIZE=IMG_SIZE):
    images = []
    labels = []
    
    method_dir = os.path.join(image_dir, method, dataset_type)
    
    # Loop through classes and load images
    for class_dir in os.listdir(method_dir):
        class_path = os.path.join(method_dir, class_dir)
        for image_file in os.listdir(class_path):
            if image_file.endswith('.png'):  # Assuming the images are in PNG format
                img_path = os.path.join(class_path, image_file)
                img = image.load_img(img_path, target_size=IMG_SIZE)
                img_array = image.img_to_array(img)  # Convert to numpy array
                img_array = img_array / 255.0  # Normalize the image
                
                images.append(img_array)
                labels.append(class_dir)  # Use class directory name as label
    
    return np.array(images), np.array(labels)

In [ ]:
# Helper function to encode labels as integers
def encode_labels(labels):
    le = LabelEncoder()
    return le.fit_transform(labels), le

#### CNN Model Definition

In [ ]:
# Build the CNN model as per the specifications
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    
    # First Convolutional Block
    model.add(layers.Conv2D(64, (1, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((1, 2)))
    
    # Second Convolutional Block
    model.add(layers.Conv2D(64, (1, 3), activation='relu'))
    model.add(layers.MaxPooling2D((1, 2)))
    
    # Third Convolutional Block 
    model.add(layers.Conv2D(128, (1, 3), activation='relu'))
    model.add(layers.MaxPooling2D((1, 2)))
    
    # Flatten and Dense Layers
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    
    # Dropout Layer
    model.add(layers.Dropout(0.1))
    
    # Output Layer with Softmax
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

#### Function to train and test the CNN for each variant

In [ ]:
def train_and_test_variant(image_dir, method, IMG_SIZE=IMG_SIZE):
    print(f"\nProcessing {method}...")

    # Load Train data
    print("Loading Train data...")
    train_images, train_labels = load_images(image_dir, method, "Train", IMG_SIZE)
    train_labels, label_encoder = encode_labels(train_labels)
    
    # Load Test data
    print("Loading Test data...")
    test_images, test_labels = load_images(image_dir, method, "Test", IMG_SIZE)
    test_labels = label_encoder.transform(test_labels)  # Transform test labels using the same encoder
    
    # Build the CNN model
    input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)  # 3 channels for RGB images
    num_classes = len(label_encoder.classes_)  # Number of classes based on label encoder
    model = build_cnn_model(input_shape, num_classes)
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    print("Training the model...")
    model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    
    # Evaluate the model on the Test data
    print("Testing the model...")
    test_loss, test_accuracy = model.evaluate(test_images, test_labels)
    print(f"Test Accuracy for {method}: {test_accuracy * 100:.2f}%")
    
    return test_accuracy

#### Train and Test the Model on all Variants

In [ ]:
# Process each variant
methods = ["Variant 1", "Variant 2", "Variant 3", "Variant 4", "Variant 5"]

# Train and test on Train and Test datasets for each variant
results = []
for method in methods:
    test_accuracy = train_and_test_variant(image_dir, method, IMG_SIZE)
    results.append({"Variant": method, "Test Accuracy (%)": test_accuracy * 100})

# Save results to an Excel file
results_df = pd.DataFrame(results)
results_df.to_excel("BasicMotions_CNN_Results.xlsx", index=False) # Change the name to currently working dataset

print("\nResults saved to 'BasicMotions_CNN_Results.xlsx'") # Change the name to currently working dataset

#### Import Libraries for Transfer Learning

In [ ]:
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, EfficientNetB3, MobileNetV2
from tensorflow.keras.optimizers import Adam


#### Function to use Transfer Learning for different models

In [ ]:

def transfer_learning(image_dir, method, model_name, base_model_func, IMG_SIZE=IMG_SIZE):
    """
    Perform transfer learning using a pre-trained model.
    """
    print(f"\nApplying Transfer Learning on {method} using {model_name}...")

    # Load Train data
    print("Loading Train data...")
    train_images, train_labels = load_images(image_dir, method, "Train", IMG_SIZE)
    train_labels, label_encoder = encode_labels(train_labels)

    # Load Test data
    print("Loading Test data...")
    test_images, test_labels = load_images(image_dir, method, "Test", IMG_SIZE)
    test_labels = label_encoder.transform(test_labels)  # Transform test labels using the same encoder

    # Load pre-trained model
    base_model = base_model_func(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

    # Freeze the base model layers
    base_model.trainable = False

    # Build the transfer learning model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    print("Training the model...")
    model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

    # Evaluate the model on the Test data
    print("Testing the model...")
    test_loss, test_accuracy = model.evaluate(test_images, test_labels)
    print(f"Transfer Learning Test Accuracy for {method} using {model_name}: {test_accuracy * 100:.2f}%")

    return test_accuracy



#### Apply Transfer Learning on all variants.

In [ ]:
# Define models and their corresponding functions
models_info = [
    ("VGG16", VGG16),
    ("VGG19", VGG19),
    ("ResNet50", ResNet50),
    ("EfficientNetB3", EfficientNetB3),
    ("MobileNetV2", MobileNetV2)
]

# Process each variant with Transfer Learning for all models
all_results = []

for model_name, base_model_func in models_info:
    print(f"\nProcessing {model_name}...")
    transfer_results = []
    for method in methods:
        test_accuracy = transfer_learning(image_dir, method, model_name, base_model_func, IMG_SIZE)
        transfer_results.append({"Model": model_name, "Variant": method, "Test Accuracy (%)": test_accuracy * 100})
    
    all_results.extend(transfer_results)

# Save transfer learning results to an Excel file
all_results_df = pd.DataFrame(all_results)
all_results_df.to_excel("BasicMotions_Transfer_Learning.xlsx", index=False) # Change the name to currently working dataset

print("\nTransfer learning results saved to 'BasicMotions_Transfer_Learning.xlsx'") # Change the name to currently working dataset